# Name: Mohammed Samir Ali 
# ID:   SW01080809

# Name: Muhammad Farish Naufal Bin Norzali
# ID: SW01081139

In [25]:

#Name: Mohammed Samir Ali
#ID:   SW01080809
#And
#Name: Muhammad Farish Naufal Bin Norzali
#ID:   SW01081139¶

import re
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

def convert_timestamp(timestamp):
    """
    Convert a timestamp to a human-readable datetime string.

    Args:
        timestamp (str): The timestamp to convert.

    Returns:
        str: A human-readable datetime string in the format '%Y-%m-%d %H:%M:%S'.
             Returns None if the conversion fails.
    """
    try:
        timestamp = int(timestamp)
        return datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
    except ValueError:
        return None

def scrape_reviews(url):
    """
    Scrape reviews for a product from Shopee.

    Args:
        url (str): The URL of the product page.

    Returns:
        list: A list of dictionaries containing review information.
              Each dictionary contains 'reviewer_name', 'review_content', and 'review_date'.
    """
    # Extract shop ID and item ID from the URL
    r = re.search(r'i\.(\d+)\.(\d+)', url)
    if r:
        shop_id, item_id = r.group(1), r.group(2)
    else:
        print("Invalid URL")
        return []

    reviews = []

    offset = 0
    while True:
        # Construct the URL for fetching reviews
        ratings_url = f'https://shopee.com.my/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'
        data = requests.get(ratings_url).json()

        # Extract review information and add to the list
        for rating in data['data']['ratings']:
            review_date = convert_timestamp(rating['ctime'])
            if review_date:
                reviews.append({
                    'reviewer_name': rating['author_username'],
                    'review_content': rating['comment'],
                    'review_date': review_date
                })

        # Break the loop if there are fewer than 20 reviews on the page
        if len(data['data']['ratings']) < 20:
            break

        offset += 20

    return reviews

def save_to_csv(reviews, filename='product_reviews.csv'):
    """
    Save review data to a CSV file.

    Args:
        reviews (list): A list of dictionaries containing review information.
        filename (str): The name of the CSV file to save the data to.
                        Default is 'product_reviews.csv'.
    """
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['reviewer_name', 'review_date', 'review_content']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for review in reviews:
            writer.writerow(review)

if __name__ == "__main__":
    # URL of the product page which is a 'KYT Helmet' 
    url = 'https://shopee.com.my/HELMET-KYT-HELLCAT-STAR-OPEN-FACE-BLACK-WHITE-READY-STOCK--i.137282859.2239192199?sp_atk=b21b7fc3-174d-4414-9424-1e2e72af66c4&xptdk=b21b7fc3-174d-4414-9424-1e2e72af66c4'
    
    # Scrape reviews
    reviews = scrape_reviews(url)
    
    # Save reviews to CSV file
    if reviews:
        save_to_csv(reviews)
        print("Reviews scraped and saved to CSV successfully.")
    else:
        print("No reviews found.")


Reviews scraped and saved to CSV successfully.
